In [ ]:
import pandas as pd
import json
import os

In [ ]:
component_to_analyze = "example"

os.system("radon cc -j {} > radon_cc.json".format(component_to_analyze))
os.system("radon mi -j {} > radon_mi.json".format(component_to_analyze))
os.system("radon raw -j {} > radon_raw.json".format(component_to_analyze))
os.system("pylint -f json {} > pylint.json".format(component_to_analyze))

radon_cc_filename = "radon_cc.json"
radon_mi_filename = "radon_mi.json"
radon_raw_filename = "radon_raw.json"
pylint_filename = "pylint.json"

In [ ]:
def append_component_cc(df, components_dict):
    for key, value in components_dict.items():
            sum_cc = 0
            sub_parts = 0
            for j in value:
                sum_cc += j["complexity"]
                sub_parts += 1
            df = df.append({'path': key,'sum_cc': sum_cc, 'sub_parts': sub_parts}, ignore_index=True)
    return df
            
def create_df_cc():
    with open(radon_cc_filename, "r") as f:
        radon_cc_json = json.load(f)
        df_cc = pd.DataFrame()
        df_cc = append_component_cc(df_cc, radon_cc_json)
        df_cc = df_cc.set_index('path')
        
        return df_cc

In [ ]:
df_cc = create_df_cc()

In [ ]:
df_mi = pd.read_json(radon_mi_filename, orient="index")

In [ ]:
df_raw = pd.read_json(radon_raw_filename, orient="index")

In [ ]:
df_pylint = pd.read_json(pylint_filename, orient="columns")
df_pylint = df_pylint.drop(columns=['column', 'line', 'message', 'module', 'obj', 'symbol'])
df_pylint = pd.get_dummies(df_pylint, columns=['message-id', 'type'])
df_pylint = df_pylint.groupby(['path']).sum()

In [ ]:
# Combine dataframes
df_complete = pd.concat([df_cc, df_mi, df_raw, df_pylint], axis=1, sort=True)

In [ ]:
df_complete